## Preprocessing

In [40]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

# Import and read the charity_data.csv.
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [41]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'
application_df = application_df.drop(columns=['EIN', 'NAME'])
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [42]:
# Determine the number of unique values in each column
unique_values = application_df.nunique()
print(unique_values)

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64


In [43]:
# Replace rare categorical values in 'APPLICATION_TYPE' with 'Other'
application_counts = application_df['APPLICATION_TYPE'].value_counts()
replace_application = list(application_counts[application_counts < 500].index)
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(replace_application, 'Other')
print(application_df['APPLICATION_TYPE'].value_counts())

APPLICATION_TYPE
T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: count, dtype: int64


In [44]:
# Replace rare categorical values in 'CLASSIFICATION' with 'Other'
classification_counts = application_df['CLASSIFICATION'].value_counts()
replace_class = list(classification_counts[classification_counts < 1000].index)
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(replace_class, 'Other')
print(application_df['CLASSIFICATION'].value_counts())

CLASSIFICATION
C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: count, dtype: int64


In [45]:
# Convert categorical data to numeric using one-hot encoding
application_df_encoded = pd.get_dummies(application_df)
application_df_encoded.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
1,1,108590,1,False,False,False,True,False,False,False,...,True,False,False,False,False,False,False,False,True,False
2,1,5000,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,True,False
3,1,6692,1,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,True,False
4,1,142590,1,False,False,False,True,False,False,False,...,False,False,True,False,False,False,False,False,True,False


In [46]:
# Split the preprocessed data into features and target arrays
y = application_df_encoded['IS_SUCCESSFUL'].values
X = application_df_encoded.drop(columns=['IS_SUCCESSFUL']).values

# Split the preprocessed data into training and testing datasets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [47]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## Compile, Train, and Evaluate the Model

In [48]:
# Define the model
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=80, input_dim=X_train_scaled.shape[1], activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/opt/anaconda3/envs/dev/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_21 (Dense)                │ (None, 80)             │         3,520 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 30)             │         2,430 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 1)              │            31 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,981 (23.36 KB)

 Trainable params: 5,981 (23.36 KB)

 Non-trainable params: 0 (0.00 B)

In [49]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [50]:
# Custom callback to save weights every 5 epochs
class CustomCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_freq):
        super(CustomCheckpoint, self).__init__()
        self.save_freq = save_freq

    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % self.save_freq == 0:
            # Correct the file extension to `.weights.h5`
            filepath = f'model_weights_epoch_{epoch + 1:02d}.weights.h5'
            self.model.save_weights(filepath)
            print(f'Saving model weights at epoch {epoch + 1}')

# Instantiate the custom callback
checkpoint_cb = CustomCheckpoint(save_freq=5)


In [51]:
# Train the model with custom checkpoint callback
history = nn.fit(X_train_scaled, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[checkpoint_cb])


Epoch 1/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 650us/step - accuracy: 0.6992 - loss: 0.5923 - val_accuracy: 0.7380 - val_loss: 0.5492
Epoch 2/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 403us/step - accuracy: 0.7247 - loss: 0.5600 - val_accuracy: 0.7383 - val_loss: 0.5511
Epoch 3/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 411us/step - accuracy: 0.7249 - loss: 0.5578 - val_accuracy: 0.7380 - val_loss: 0.5495
Epoch 4/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 359us/step - accuracy: 0.7299 - loss: 0.5489 - val_accuracy: 0.7396 - val_loss: 0.5485
Epoch 5/100
685/686 ━━━━━━━━━━━━━━━━━━━━ 0s 293us/step - accuracy: 0.7285 - loss: 0.5550Saving model weights at epoch 5
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 407us/step - accuracy: 0.7286 - loss: 0.5550 - val_accuracy: 0.7363 - val_loss: 0.5493
Epoch 6/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 363us/step - accuracy: 0.7356 - loss: 0.5465 - val_accuracy: 0.7371 - val_loss: 0.5453
Epoch 7/100
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 361us/step - accuracy: 0.7296 - loss: 0.5484 - val_accuracy: 0

In [52]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

215/215 - 0s - 400us/step - accuracy: 0.7270 - loss: 0.5687
Loss: 0.5687134265899658, Accuracy: 0.7269679307937622


In [53]:
# Export the model to an HDF5 file
nn.save('AlphabetSoupCharity.h5')

## Step 3: Optimize the Model #1

In [54]:
# Model Optimization - Attempt 1: Add another hidden layer
nn_opt = tf.keras.models.Sequential()

# Add more neurons and an additional hidden layer
nn_opt.add(tf.keras.layers.Dense(units=100, input_dim=X_train_scaled.shape[1], activation='relu'))
nn_opt.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn_opt.add(tf.keras.layers.Dense(units=25, activation='relu'))
nn_opt.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the optimized model
nn_opt.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the optimized model
history_opt = nn_opt.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2)

# Evaluate the optimized model
model_loss_opt, model_accuracy_opt = nn_opt.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Loss: {model_loss_opt}, Optimized Accuracy: {model_accuracy_opt}")

# Save the optimized model
nn_opt.save('AlphabetSoupCharity_Optimization.h5')

Epoch 1/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 519us/step - accuracy: 0.6974 - loss: 0.5922 - val_accuracy: 0.7376 - val_loss: 0.5483
Epoch 2/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 401us/step - accuracy: 0.7209 - loss: 0.5637 - val_accuracy: 0.7378 - val_loss: 0.5531
Epoch 3/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 447us/step - accuracy: 0.7324 - loss: 0.5462 - val_accuracy: 0.7382 - val_loss: 0.5516
Epoch 4/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 434us/step - accuracy: 0.7306 - loss: 0.5486 - val_accuracy: 0.7349 - val_loss: 0.5506
Epoch 5/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 524us/step - accuracy: 0.7337 - loss: 0.5464 - val_accuracy: 0.7358 - val_loss: 0.5505
Epoch 6/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 520us/step - accuracy: 0.7294 - loss: 0.5481 - val_accuracy: 0.7396 - val_loss: 0.5454
Epoch 7/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 456us/step - accuracy: 0.7241 - loss: 0.5511 - val_accuracy: 0.7387 - val_loss: 0.5443
Epoch 8/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 467us/step - accuracy: 0.7373 - loss: 0

Optimized Loss: 0.5991957783699036, Optimized Accuracy: 0.725218653678894


Optimize Model #2


In [55]:
# Model Optimization - Attempt 2: Change Activation Functions and Increase Epochs
nn_opt_2 = tf.keras.models.Sequential()

# Add more neurons and change the activation functions
nn_opt_2.add(tf.keras.layers.Dense(units=100, input_dim=X_train_scaled.shape[1], activation='tanh'))
nn_opt_2.add(tf.keras.layers.Dense(units=50, activation='tanh'))
nn_opt_2.add(tf.keras.layers.Dense(units=25, activation='tanh'))
nn_opt_2.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the optimized model
nn_opt_2.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the optimized model with more epochs
history_opt_2 = nn_opt_2.fit(X_train_scaled, y_train, epochs=200, batch_size=32, validation_split=0.2)

# Evaluate the optimized model
model_loss_opt_2, model_accuracy_opt_2 = nn_opt_2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Loss 2: {model_loss_opt_2}, Optimized Accuracy 2: {model_accuracy_opt_2}")

# Save the optimized model
nn_opt_2.save('AlphabetSoupCharity_Optimization_2.h5')


Epoch 1/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 518us/step - accuracy: 0.7127 - loss: 0.5830 - val_accuracy: 0.7362 - val_loss: 0.5508
Epoch 2/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 446us/step - accuracy: 0.7275 - loss: 0.5558 - val_accuracy: 0.7374 - val_loss: 0.5534
Epoch 3/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 408us/step - accuracy: 0.7306 - loss: 0.5552 - val_accuracy: 0.7376 - val_loss: 0.5462
Epoch 4/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 471us/step - accuracy: 0.7301 - loss: 0.5536 - val_accuracy: 0.7298 - val_loss: 0.5494
Epoch 5/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 453us/step - accuracy: 0.7315 - loss: 0.5473 - val_accuracy: 0.7403 - val_loss: 0.5450
Epoch 6/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step - accuracy: 0.7295 - loss: 0.5502 - val_accuracy: 0.7396 - val_loss: 0.5445
Epoch 7/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.7289 - loss: 0.5471 - val_accuracy: 0.7383 - val_loss: 0.5445
Epoch 8/200
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 468us/step - accuracy: 0.7339 - loss: 0

Optimized Loss 2: 0.5746605396270752, Optimized Accuracy 2: 0.7221574187278748


Optimize Model #3

In [56]:
# Model Optimization - Attempt 3: Add Dropout Layers
nn_opt_3 = tf.keras.models.Sequential()

# Add more neurons, additional hidden layers, and dropout
nn_opt_3.add(tf.keras.layers.Dense(units=100, input_dim=X_train_scaled.shape[1], activation='relu'))
nn_opt_3.add(tf.keras.layers.Dropout(0.2))  # Drop 20% of the neurons randomly
nn_opt_3.add(tf.keras.layers.Dense(units=50, activation='relu'))
nn_opt_3.add(tf.keras.layers.Dropout(0.2))
nn_opt_3.add(tf.keras.layers.Dense(units=25, activation='relu'))
nn_opt_3.add(tf.keras.layers.Dropout(0.2))
nn_opt_3.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Compile the optimized model
nn_opt_3.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the optimized model
history_opt_3 = nn_opt_3.fit(X_train_scaled, y_train, epochs=150, batch_size=32, validation_split=0.2)

# Evaluate the optimized model
model_loss_opt_3, model_accuracy_opt_3 = nn_opt_3.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Optimized Loss 3: {model_loss_opt_3}, Optimized Accuracy 3: {model_accuracy_opt_3}")

# Save the optimized model
nn_opt_3.save('AlphabetSoupCharity_Optimization_3.h5')


Epoch 1/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 1s 597us/step - accuracy: 0.6727 - loss: 0.6200 - val_accuracy: 0.7341 - val_loss: 0.5560
Epoch 2/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step - accuracy: 0.7161 - loss: 0.5791 - val_accuracy: 0.7385 - val_loss: 0.5527
Epoch 3/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - accuracy: 0.7236 - loss: 0.5626 - val_accuracy: 0.7385 - val_loss: 0.5514
Epoch 4/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 489us/step - accuracy: 0.7246 - loss: 0.5629 - val_accuracy: 0.7372 - val_loss: 0.5454
Epoch 5/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - accuracy: 0.7273 - loss: 0.5590 - val_accuracy: 0.7389 - val_loss: 0.5459
Epoch 6/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 448us/step - accuracy: 0.7315 - loss: 0.5568 - val_accuracy: 0.7358 - val_loss: 0.5484
Epoch 7/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 445us/step - accuracy: 0.7339 - loss: 0.5521 - val_accuracy: 0.7383 - val_loss: 0.5479
Epoch 8/150
686/686 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - accuracy: 0.7293 - loss: 0

Optimized Loss 3: 0.5627670884132385, Optimized Accuracy 3: 0.7265306115150452
